In [6]:
def read_mtx_file(file_path):
    with open(file_path, 'r') as file:
        # Read the lines from the file
        lines = file.readlines()

        # Extract matrix dimensions and non-zero entries
        rows, columns, non_zeros = map(int, lines[1].split())

        # Extract non-zero entries
        non_zero_entries = [line.strip().split() for line in lines[2:]]

    return rows, columns, non_zeros, non_zero_entries

def write_to_txt(rows, columns, non_zeros, non_zero_entries, output_file):
    with open(output_file, 'w') as file:
        # Write matrix dimensions and non-zero entries to the txt file
        file.write(f"{rows} {columns} {non_zeros}\n")

        # Write non-zero entries to the txt file
        for entry in non_zero_entries:
            file.write(" ".join(entry) + "\n")

# Replace 'input.mtx' and 'output.txt' with your file names
input_file = 'links.mtx'
output_file = 'output.txt'

# Read the MatrixMarket file
rows, columns, non_zeros, non_zero_entries = read_mtx_file(input_file)

# Write the data to a text file
write_to_txt(rows, columns, non_zeros, non_zero_entries, output_file)

In [7]:
import numpy as np

# Initialize a matrix of zeros
matrix_size = rows
matrix = np.zeros((matrix_size, matrix_size))

# Read data from the text file
with open('output.txt', 'r') as file:
    lines = file.readlines()
    lines.remove(lines[0])

check = 0

# Process each line in the file
for line in lines:
    # Split the line into two values
    values = line.split()
    col_index = int(values[0]) - 1  # Convert to 0-based index
    row_index = int(values[1]) - 1  # Convert to 0-based index

    # check = check+1

    # Increment the corresponding element in the matrix
    matrix[row_index, col_index] += 1

output_file_path = 'matrix.txt'
np.savetxt(output_file_path, matrix, fmt='%d')

print(f"Matrix has been saved to {output_file_path}")


Matrix has been saved to matrix.txt


In [8]:
print(np.count_nonzero(matrix))

6474


In [9]:
# Normalize the columns of the matrix for non-zero values
non_zero_columns = np.sum(matrix, axis=0) != 0
matrix[:, non_zero_columns] /= np.sum(matrix[:, non_zero_columns], axis=0)

# Print the normalized matrix
print(matrix)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [10]:
import numpy as np

# Find columns with only zero elements
zero_columns = np.where(~matrix.any(axis=0))[0]

# Replace columns with only zero elements with 1/3031
matrix[:, zero_columns] = 1/3031

column_sums = []

# Verify that the sum of each column is 1
column_sums = matrix.sum(axis=0)

print(column_sums)

# Verify that the matrix is column stochastic
print("Is Column Stochastic:", np.allclose(column_sums, 1))


[1. 1. 1. ... 1. 1. 1.]
Is Column Stochastic: True


In [11]:
print(np.count_nonzero(matrix))

1134006


In [12]:
# Check if the sum of each column is approximately equal to 1
column_sums = matrix.sum(axis=0)
is_column_stochastic = np.allclose(column_sums, 1)

if is_column_stochastic:
    print("The matrix is column stochastic.")
else:
    print("The matrix is not column stochastic.")

The matrix is column stochastic.


In [13]:
L = matrix
print(L)

[[3.29924117e-04 3.29924117e-04 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [3.29924117e-04 3.29924117e-04 1.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [3.29924117e-04 3.29924117e-04 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [3.29924117e-04 3.29924117e-04 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [3.29924117e-04 3.29924117e-04 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [3.29924117e-04 3.29924117e-04 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]


In [14]:
import numpy as np

def page_rank(matrix, damping_factor=0.85, epsilon=1e-8, max_iterations=1000):
    """
    PageRank algorithm implementation.

    Parameters:
    - matrix: The column stochastic matrix representing the transition probabilities.
    - damping_factor: The damping factor (typically set to 0.85).
    - epsilon: Convergence criterion.
    - max_iterations: Maximum number of iterations.

    Returns:
    - page_ranks: A 1D array containing the PageRank scores for each vertex.
    """

    # Get the number of vertices
    num_vertices = matrix.shape[0]

    # Initialize PageRank scores
    page_ranks = np.ones(num_vertices) / num_vertices

    for _ in range(max_iterations):
        # Store the current PageRank scores for convergence check
        old_page_ranks = page_ranks.copy()

        # Update PageRank scores
        page_ranks = (1 - damping_factor) / num_vertices + damping_factor * np.dot(matrix, page_ranks)

        # Check for convergence
        if np.linalg.norm(page_ranks - old_page_ranks, 1) < epsilon:
            break

    return page_ranks

# Assuming you have the column stochastic matrix L
# Compute PageRank scores
page_ranks = page_rank(L)

# Print the results
print("PageRank Scores:")
print(page_ranks)


PageRank Scores:
[4.60150052e-04 9.02848998e-04 7.21721644e-04 ... 6.72009094e-05
 6.72009094e-05 6.72009094e-05]


In [15]:
print(matrix)

[[3.29924117e-04 3.29924117e-04 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [3.29924117e-04 3.29924117e-04 1.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [3.29924117e-04 3.29924117e-04 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [3.29924117e-04 3.29924117e-04 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [3.29924117e-04 3.29924117e-04 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [3.29924117e-04 3.29924117e-04 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]


In [16]:
# Rank the vertices in descending order
ranked_vertices = np.argsort(page_ranks)[::-1]

# Add 1 to each ranked vertex as we assumed a zero indexed graph
ranked_vertices = ranked_vertices + 1

print("Ranked Vertices:", ranked_vertices)

Ranked Vertices: [  16 2907 2965 ... 3031  380  379]


In [17]:
# Top n imp vertices

top_n = 5  # Change this to the desired number of top vertices
top_vertices = ranked_vertices[:top_n]
print("Top", top_n, "Vertices:", top_vertices)


Top 5 Vertices: [  16 2907 2965  342 2908]
